In [136]:
from decimal import *
import math
from curve_shift import CNT_SEG, alpha, RONE

In [137]:
getcontext().prec = 25

swapFee = Decimal(0)
swapFee = Decimal(0.0035)
afterSwapFee = Decimal(1) - swapFee
protocolFee = Decimal(0.2)
totalSupply = Decimal(0)
MINIMUM_LIQUIDITY = 10**3

In [138]:
def getWeight(T):
    res = alpha[T//3600]
    return (Decimal(RONE - (res * RONE)), Decimal((res * RONE)))

def getRawWeight(T):
    res = alpha[T//3600]
    return (Decimal(1-res), Decimal(res))


In [139]:
################ K STUFF #################################
kLast = -1
totalTreasury = 0

def calcK(T):
    Win, Wout = getRawWeight(T)
    global Bin, Bout
    return (pow(Bin, Win) * pow(Bout, Wout))

def updateK(T):
    global kLast
    kNow = calcK(T)
    kLast = kNow

def updateTreasury(T):
    global kLast, totalSupply, totalTreasury
    kNow = calcK(T)
    gainedLp = totalSupply * (kNow - kLast) / ((1/protocolFee - 1) * kNow + kLast)
    totalTreasury += gainedLp
    totalSupply += gainedLp
    updateK(T)

#######################################################




    

def updateBalance(da, db, xytInput):
    if xytInput:
        dc = da
        da = db
        db = dc
    
    global Bin, Bout
    Bin = Bin + da
    Bout = Bout + db

def calOutWeight(T, Bin, Bout, inAmount, xytInput=True):
    Win, Wout = getWeight(T)
    oldInAmount = inAmount
    inAmount = inAmount * afterSwapFee
    if xytInput:
        Wout, Win = getWeight(T)
        temp = Bin
        Bin = Bout
        Bout = temp
    res = Bout * (1 - pow((Bin / (Bin + inAmount)), Win / Wout))
    return res

def swapTokenToXyt(T, inAmount):
    updateK(T)
    global Bin, Bout
    outAmount = calOutWeight(T, Bin, Bout, inAmount, False)
    updateBalance(inAmount, -outAmount, False)
    updateTreasury(T)


def swapXytToToken(T, inAmount):
    updateK(T)
    global Bin, Bout
    outAmount = calOutWeight(T, Bin, Bout, inAmount, True)
    updateBalance(inAmount, -outAmount, True)
    updateTreasury(T)

def addLiquidityDual(T, amount):
    x = amount
    y = amount
    global Bin, Bout, totalSupply
    if Bin < Bout:
        x = amount * Bin / Bout
    else:
        y = amount * Bout / Bin
    Bin += x
    Bout += y
    totalSupply += totalSupply * x / Bin
    updateK(T)

In [140]:
def run_swap(time, swapTime):
    for i in range(swapTime):
        global Bin, Bout
        amount = Decimal(40000000 + 5000000 * i)
        if i % 3 == 0:
            calOutWeight(time, Bin, Bout, amount, False)
        else:
            calOutWeight(time, Bin, Bout, amount, True)
        addLiquidityDual(amount)

In [141]:
Bin = Decimal(1000000000)
Bout = Decimal(1000000000)

T = 3600
constsSwapAmount = Decimal(100000000)
totalSupply = Decimal.sqrt(Bin * Bout)

print(getWeight(T))


swapXytToToken(T, constsSwapAmount)
swapTokenToXyt(T, constsSwapAmount)
addLiquidityDual(T, constsSwapAmount)
print(totalTreasury, totalSupply)


swapXytToToken(T, constsSwapAmount)
swapTokenToXyt(T, constsSwapAmount)
addLiquidityDual(T, constsSwapAmount)
print(totalTreasury, totalSupply)

swapXytToToken(T, constsSwapAmount)
swapTokenToXyt(T, constsSwapAmount)
addLiquidityDual(T, constsSwapAmount)
print(totalTreasury, totalSupply)

swapXytToToken(T, constsSwapAmount)
swapTokenToXyt(T, constsSwapAmount)
addLiquidityDual(T, constsSwapAmount)
print(totalTreasury, totalSupply)


print("1 hour precision:")
print(66497/66501)
print(133298/132764)
print(198826/200350)


# # firstK = getK(0)
# # exit(0)

# addLiquidityDual(Decimal(1000000000))


# print(calOutWeight(3600, Bin, Bout, 
#     Decimal(220000), 
#     False
# ))
# updateK(3600 + 3600)
# # exit(0)

# print(calOutWeight(36000, Bin, Bout, 
#     Decimal(25000),
#     True
# ))

# run_swap(36000+3600, 10)
# updateK(36000+3600)

# print(calOutWeight(360000, Bin, Bout, 
#     Decimal(725000),
#     True
# ))
# updateK(360000+3600)

# print(calOutWeight(1692000, Bin, Bout,
#     Decimal(400000),
#     False
# ))
# updateK(1692000+3600)

# run_swap(1692000+3600+3600, 10)
# updateK(1692000+3600+3600+3600)


# # lastK = getK(1692000)

# # print(firstK, lastK)

(Decimal('549789788090.22021484375'), Decimal('549721839685.77978515625'))
66501.58997744866912510973 1090212046.440332679622245
Ratio: 0.00006099876642767247395683360
132764.2310541277744637679 1179906851.775972742679583
198826.4532663120285775666 1269246270.344687049521742
264716.7715118707285406099 1358339853.404427671059991
